<a href="https://colab.research.google.com/github/Vizzuality/mangrove-atlas-data/blob/master/process_osm_coastline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data for the mangrove-atlas project

https://github.com/Vizzuality/mangrove-atlas-data

`Edward P. Morris (vizzuality.)`

## Description
This notebook gets the latest OSM coastline dataset and uploads it to earthengine. 

```
MIT License

Copyright (c) 2020 Vizzuality

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

# Setup

Instructions for setting up the computing environment.

In [ ]:
%%bash
# Remove sample_data
rm -r sample_data

## Linux dependencies

Instructions for adding linux (including node, ect.) system packages. 

In [ ]:
%%bash
#apt install -q -y [package-name]
#npm install -g [package-name]

## Python packages

In [ ]:
%%bash
#pip install -q [package-name]

In [ ]:
# Show python package versions
#!pip list

## Authorisation

Setting up connections and authorisation to cloud services.

### Google Cloud

This can be done in the URL or via adding service account credentials.

If you do not share the notebook, you can mount your Drive and and transfer credentials to disk. Note if the notebook is shared you always need to authenticate via URL.  

In [ ]:
# Set the Google Cloud project id
project_id = "mangrove-atlas-246414"
gc_creds = "mangrove-atlas-246414-2f33cc439deb.json"
username = "edward-morris-vizzuality-com-d@mangrove-atlas-246414.iam.gserviceaccount.com"
gcs_prefix = "gs://mangrove_atlas"

In [ ]:
# For auth WITHOUT service account
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#from google.colab import auth
#auth.authenticate_user()
#!gcloud config set project {project_id}

In [ ]:
# If the notebook is shared
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# If Drive is mounted, copy GC credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/{gc_creds}" "/root/.{gc_creds}"

In [ ]:
# Auth WITH service account
!gcloud auth activate-service-account {username} --key-file=/root/.{gc_creds} --project={project_id}

Activated service account credentials for: [edward-morris-vizzuality-com-d@mangrove-atlas-246414.iam.gserviceaccount.com]


In [ ]:
# Test GC auth
!gsutil ls {gcs_prefix}

gs://mangrove_atlas/wdpa_geometry_types_.csv
gs://mangrove_atlas/./
gs://mangrove_atlas//
gs://mangrove_atlas/boundaries/
gs://mangrove_atlas/deforestation-alerts/
gs://mangrove_atlas/ee-export-tables/
gs://mangrove_atlas/ee-upload-manifests/
gs://mangrove_atlas/elevation/
gs://mangrove_atlas/environmental-pressures/
gs://mangrove_atlas/gadm-eez.zarr/
gs://mangrove_atlas/land-cover/
gs://mangrove_atlas/mangrove-properties/
gs://mangrove_atlas/orthoimagery/
gs://mangrove_atlas/physical-environment/
gs://mangrove_atlas/tilesets/
gs://mangrove_atlas/tmp/


# Utils

Generic helper functions used in the subsequent processing. For easy navigation each function seperated into a section with the function name.

## copy_gcs

In [ ]:
import os
import subprocess

def copy_gcs(source_list, dest_list, opts=""):
  """
  Use gsutil to copy each corresponding item in source_list
  to dest_list.

  Example:
  copy_gcs(["gs://my-bucket/data-file.csv"], ["."])

  """
  for s, d  in zip(source_list, dest_list):
    cmd = f"gsutil -m cp -r {opts} {s} {d}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
  print("Finished copy")

## gee_upload_zip_to_table

In [ ]:
def gee_upload_zip_to_table(file_path, asset_path, time_start, force=True, append_txt="", properties={}):
    '''Given a GCS file path and a GEE asset path
       Create, and push to all zipped shapefiles + metadata to MANY Table assets
    '''
    # Format arguments
    f = ""
    if force:
      f = "--force"
    ts = f"--time_start={time_start}"
    p = ""
    if len(properties) > 0:
      p = ["--property='{0}={1}'".format(key, value) for key, value in properties.items()]
      p = " ".join(p) 
    args = "{0} {1} {2} ".format(f, ts, p)
    
    asset_id = os.path.splitext(os.path.basename(file_path))[0]
    print("Processing {0}".format(asset_id))
    cmd = "earthengine --no-use_cloud_api upload table --asset_id={0}/{1}{2} {3} {4}".format(asset_path, asset_id, append_txt, args, file_path)
    print(cmd)
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished upload")

## gee_update_asset_properties

In [ ]:
import subprocess
import json

def gee_update_asset_properties(asset_path, properties = {}, time_start=None, time_end=None, dry_run=False):
  
  # Format arguments
  ts = ""
  if time_start:
    ts = f"--time_start={time_start}"
  te = ""
  if time_end:
    te = f"--time_end={time_end}"  
  p = ""
  if len(properties) > 0:
    p = [f"--property={key}={json.dumps(value)}" for key, value in properties.items()]
    p = " ".join(p) 
  args = f"{ts} {te} {p}"

  # Update asset
  cmd = f"earthengine --no-use_cloud_api asset set {args} {asset_path}"
  if dry_run:
    print(cmd)
  else:
    r = subprocess.call(cmd, shell=True)
    if r == 0:
      print(f"\nUpdated properties for asset: {asset_path}\n")
      cmd = f"earthengine --no-use_cloud_api asset info {asset_path}"
      out = subprocess.check_output(cmd, shell=True).decode('utf8')
      print(out)
    else:
      print("Task failed")
      print(cmd)


## folium_add_ee_layer

In [ ]:
# Import libraries.
import ee
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Processing

Data processing organised into sections.

## Get datasets

In [ ]:
import requests, zipfile, io
file_url = "https://osmdata.openstreetmap.de/download/coastlines-split-4326.zip"
r = requests.get(file_url, stream = True) 
  
with open("coastlines-split-4326.zip","wb") as f: 
    for chunk in r.iter_content(chunk_size=1024): 
  
         # writing one chunk at a time to pdf file 
         if chunk: 
             f.write(chunk) 

In [ ]:
copy_gcs(["coastlines-split-4326.zip"], [f"{gcs_prefix}/physical-environment"])

Processing: gsutil -m cp -r  coastlines-split-4326.zip gs://mangrove_atlas/physical-environment
Task created
Finished copy


## Create OSM coastlines feature collection

### Upload to earthengine

In [ ]:
!earthengine --no-use_cloud_api authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tmjRBNysgRYfmEhf20TosoFPrUWSBTyTVZifK2qYOtU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1QHPlACI0-r_pB503K9zMlowJ_SqleF0_zxPT_vyPUVWGM88tGyz450

Successfully saved authorization token.


In [ ]:
file_path = f"{gcs_prefix}/physical-environment/coastlines-split-4326.zip"
asset_path = "projects/global-mangrove-watch/physical-environment"
time_start = "2020-06-29"
gee_upload_zip_to_table(file_path, asset_path, time_start, force=True, append_txt="", properties={})

Processing coastlines-split-4326
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/physical-environment/coastlines-split-4326 --force --time_start=2020-06-29   gs://mangrove_atlas/physical-environment/coastlines-split-4326.zip
Task created
Finished upload


### Add metadata

In [ ]:
!earthengine --no-use_cloud_api authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=aPh6B2Y7wyIAIieyL49q5UH56OrlG43IwoUZeL9h0ms&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1gH9HvICcy1oBBQ_fXZdY-wSR0fuSU8CWGfm6CWYzPNHclDvgrmEc8k

Successfully saved authorization token.


In [ ]:
# Define parameters

# asset path
asset_path = f"projects/global-mangrove-watch/physical-environment/coastlines-split-4326"

# get description from file
copy_gcs(["gs://mangrove_atlas/physical-environment/coastlines-split-4326_description.md"], ["."])
with open("coastlines-split-4326_description.md", "r") as f:
  description = f.read()

# set collection properties (these are compatible with Skydipper.Dataset.Metadata)
version = "2020-06-29"
collection_properties = {
    'name': "Open Street Map Coastlines",
    'version': version,
    'creator': "OSM",
    'system:description': description,
    'description': description,
    'identifier': "",
    'keywords': "Erosion; Coasts; Natural Infrastructure; Sea-level; Vector",
    'citation': "",
    'license': "[Open Database License (ODbL)](https://wiki.osmfoundation.org/wiki/Licence)",
    'url': "https://osmdata.openstreetmap.de",
    'language': 'en', 
    'altName': f"OSM Coastlines, Version {version}",
    'distribution': "https://osmdata.openstreetmap.de/download/coastlines-split-4326.zip",
    'variableMeasured': "Linestrings representing coastlines",
    'units': "1",
    'spatialCoverage': "Global",
    'temporalCoverage': "undefined",
    'dataLineage': "Zipped shapefile in EPSG WSG84 downloaded from https://osmdata.openstreetmap.de, and added to Google earth engine as a featureCollection."
}

# set start and end times
time_start = '2020-06-29'
time_end = '2020-06-29'

Processing: gsutil -m cp -r  gs://mangrove_atlas/physical-environment/coastlines-split-4326_description.md .
Task created
Finished copy


In [ ]:
gee_update_asset_properties(asset_path, properties = collection_properties, time_start=time_start, time_end=time_end, dry_run=False)


Updated properties for asset: projects/global-mangrove-watch/physical-environment/coastlines-split-4326

{
  "columns": {
    "FID": "Long",
    "system:index": "String"
  },
  "id": "projects/global-mangrove-watch/physical-environment/coastlines-split-4326",
  "properties": {
    "altName": "OSM Coastlines, Version 2020-06-29",
    "citation": "",
    "creator": "OSM",
    "dataLineage": "Zipped shapefile in EPSG WSG84 downloaded from https://osmdata.openstreetmap.de, and added to Google earth engine as a featureCollection.",
    "description": "# Open Street Map Coastlines\\n\\nThis dataset shows linestrings representing coastlines extracted from the Open Street Map project.\\n\\nThe dataset has been generated by vizzuality on behalf of Global Mangrove Watch (GMW) \\u2013 a collaboration between Aberystwyth University (U.K.), solo Earth Observation (soloEO), Wetlands International, the World Conservation Monitoring Centre (UNEP-WCMC) and the Japan Aerospace Exploration Agency (JAXA)

## Create OSM coastline raster image

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=VL13F6Cu89XYt8coeUyEWJyH_jUPsggylJkxxFb-37Y&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1gE7kxA4jZKCCTkMOuY9PNfEwti2Xhg07VpsEg4-JY16-ZO82jmHzfE

Successfully saved authorization token.


In [ ]:
# Initialize the library.
ee.Initialize()

In [ ]:
# Define parameters

# asset path
asset_path = f"projects/global-mangrove-watch/physical-environment/coastlines-split-4326-raster"

# get description from file
copy_gcs(["gs://mangrove_atlas/physical-environment/coastlines-split-4326-raster_description.md"], ["."])
with open("coastlines-split-4326-raster_description.md", "r") as f:
  description = f.read()

# set collection properties (these are compatible with Skydipper.Dataset.Metadata)
version = "2020-06-29"
# set start and end times
time_start = '2020-06-29'
time_end = '2020-06-29'
# image properties
image_properties = {
    'system:id': 'coastlines-split-4326-raster',
    'system:time_start': time_start,
    'nominal_scale_m': 30,
    'band_names': 'lc',
    'band_pyramiding_policies': 'mode',
    'name': "Open Street Map Coastlines Raster",
    'version': version,
    'creator': "OSM/vizzuality",
    'system:description': description,
    'description': description,
    'identifier': "",
    'keywords': "Erosion; Coasts; Natural Infrastructure; Sea-level; Vector",
    'citation': "",
    'license': "[Open Database License (ODbL)](https://wiki.osmfoundation.org/wiki/Licence)",
    'url': "https://osmdata.openstreetmap.de",
    'language': 'en', 
    'altName': f"OSM Coastlines Raster, Version {version}",
    'distribution': "https://osmdata.openstreetmap.de/download/coastlines-split-4326.zip",
    'variableMeasured': "Boolean raster representing coastlines",
    'units': "1",
    'spatialCoverage': "Global",
    'temporalCoverage': "undefined",
    'dataLineage': "Zipped shapefile in EPSG WSG84 downloaded from https://osmdata.openstreetmap.de, and added to Google earth engine as a featureCollection. Vizzuality converted linestrings to raster at a scale of 30m using Google earth-engine using ee.FeatureCollection.reduceToImage."
}

Processing: gsutil -m cp -r  gs://mangrove_atlas/physical-environment/coastlines-split-4326-raster_description.md .
Task created
Finished copy


In [ ]:
# Create boolean raster representing coastlines

# Import libraries.
import ee
import folium
import json
import pprint

def app(debug):

  # GET DATA LAYERS
  coastline = ee.FeatureCollection('projects/global-mangrove-watch/physical-environment/coastlines-split-4326')

  # SET EXPORT REGION
  region = ee.Geometry.Rectangle([-180, -90, 180, 90], None, False)

  # CALCULATIONS
  def to_raster(fc, feature_properties, reducer, timestamp, image_properties, debug):

    # convert to raster
    im = fc \
    .reduceToImage(**{
          'properties': feature_properties,
          'reducer': reducer
      }) \
    .selfMask() \
    .rename(image_properties.get('band_names')) \
    .set(image_properties)

    # Export params
    nm = image_properties.get('system:id')
    ns = image_properties.get('nominal_scale_m')
    
    # EXPORT TO IMAGE COLLECTION
    params = {
        'image': im,
        'description': "export_" + nm,
        'assetId': 'projects/global-mangrove-watch/physical-environment/' + nm,
        'pyramidingPolicy':{image_properties.get('band_names'):image_properties.get('band_pyramiding_policies')},
        'scale': ns,
        'crs': 'EPSG:4326',
        'region': region,
        'maxPixels': 1e13
      
    }
    task = ee.batch.Export.image.toAsset(**params)
    if debug == False:
        task.start()
    
    if debug == True:
      print('\n############')
      print('\nName:', nm)
      print('\nNominal scale:', ns)
      print('\nImage:', json.dumps(im.getInfo(), indent=4))
      print("\nExport params:\n")
      pprint.pprint(params, indent=4)
      
      print("\n Map:\n")
      # Create a folium map object.
      my_map = folium.Map(location=[-7.998, 39.4767], zoom_start=9, height=500)
      # Set visualization parameters.
      vis_params = {'min': 0, 'max': 1, 'palette': ['#0000FF']}
      # Add map layer
      my_map.add_ee_layer(im, vis_params, nm)
      # Add a layer control panel to the map.
      my_map.add_child(folium.LayerControl())
      # Display the map.
      display(my_map)  
    
    return 'true'
  
  # Do the calcs
  to_raster(coastline, ee.List(['FID']), ee.Reducer.anyNonZero(), '2020-06-29', image_properties, debug)
    

# Run process
debug = False
app(debug)